## LangGraph Introduction and Basics

In [ ]:
from dotenv import load_dotenv

load_dotenv()

import os
print(os.environ['OPENAI_API_KEY'][:20])

## Simple non-AI graph example
Generate some random values, and sum them up.

In [ ]:
import random
values = [random.randint(0, 10) for _ in range(10)]
print(f"Sum of values: {sum(values)}")

In [ ]:
# Define the state of the graph:
from typing import TypedDict

class State(TypedDict):
    num_values: int
    generated_values: list[int]
    result: int | None = None


In [ ]:
import random

# Define the nodes in the graph:
def generate_values(state: State):
    values = [random.randint(0, 10) for _ in range(state["num_values"])]
    return {"generated_values": values}

def add(state: State):
    result = sum(state["generated_values"])
    return {"result": result}

In [ ]:
# Define the graph:
from langgraph.graph import StateGraph
from langgraph.graph import START, END

graph_builder = StateGraph(State)
graph_builder.add_node("generate_values", generate_values)
graph_builder.add_node("add", add)

graph_builder.add_edge(START, "generate_values")
graph_builder.add_edge("generate_values", "add")
graph_builder.add_edge("add", END)

# Now graph is also a 'Runnable'
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({"num_values": 10})

In [ ]:
print(result)

In [ ]:
await graph.ainvoke({"num_values": 10})

## Conditional Edge example in LangGraph

In [ ]:
class ConditionalEdgeState(TypedDict):
    num_values: int
    generated_values: list[int]
    total: int
    # Prints happy if the total is even, sad if the total is odd
    final_message: str

def generate_values(state: ConditionalEdgeState):
    generated_values = [random.randint(0, 10) for _ in range(state["num_values"])]
    return {"generated_values": generated_values}

def add(state: ConditionalEdgeState):
    total = sum(state["generated_values"])
    return {"total": total}

def check_total(state: ConditionalEdgeState):
    if state["total"] % 2 == 0:
        return "happy_message"
    else:
        return "sad_message"
    
def happy_message(state: ConditionalEdgeState):
    return {"final_message": "happy"}

def sad_message(state: ConditionalEdgeState):
    return {"final_message": "sad"}

graph_builder = StateGraph(ConditionalEdgeState)

graph_builder.add_node("generate_values", generate_values)
graph_builder.add_node("add", add)
graph_builder.add_node("happy_message", happy_message)
graph_builder.add_node("sad_message", sad_message)

graph_builder.add_edge(START, "generate_values")
graph_builder.add_edge("generate_values", "add")
# THIS is the place where the condition is evaluated
graph_builder.add_conditional_edges(
    "add",
    check_total,
    {
        "happy_message": "happy_message",
        "sad_message": "sad_message",
    },
)
graph_builder.add_edge("happy_message", END)
graph_builder.add_edge("sad_message", END)

graph = graph_builder.compile()


In [ ]:
# Display the graph
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"num_values": 10})

## Using LangChain Components in LangGraph
A simple sentiment analyzer that branches to happy or sad responses.


In [ ]:
# Import LangChain components
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize model and parser (can be used in LangGraph nodes!)
# llm = init_chat_model("gpt-4o-mini", model_provider="openai")
llm = init_chat_model("gpt-5-mini", model_provider="openai", reasoning_effort="minimal")
parser = StrOutputParser()


In [ ]:
# Define state
class SentimentState(TypedDict):
    sentence: str
    sentiment: str
    message: str

# Node: Analyze sentiment
def analyze_sentiment(state: SentimentState):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Analyze sentiment. Reply with ONLY 'happy' or 'sad'."),
        ("user", "{sentence}")
    ])
    chain = prompt | llm | parser
    sentiment = chain.invoke({"sentence": state["sentence"]}).strip().lower()
    return {"sentiment": sentiment}

# Node: Generate happy message
def happy_response(state: SentimentState):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Generate a cheerful, uplifting message in 1 sentence."),
        ("user", "The person said: {sentence}")
    ])
    chain = prompt | llm | parser
    message = chain.invoke({"sentence": state["sentence"]})
    return {"message": message}

# Node: Generate sad message
def sad_response(state: SentimentState):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Generate a comforting message in 1 sentence that says it's alright."),
        ("user", "The person said: {sentence}")
    ])
    chain = prompt | llm | parser
    message = chain.invoke({"sentence": state["sentence"]})
    return {"message": message}


In [ ]:
# Routing function
def route_sentiment(state: SentimentState):
    if "happy" in state["sentiment"]:
        return "happy"
    else:
        return "sad"

# Build the graph
builder = StateGraph(SentimentState)
builder.add_node("analyze_sentiment", analyze_sentiment)
builder.add_node("happy", happy_response)
builder.add_node("sad", sad_response)

builder.add_edge(START, "analyze_sentiment")
builder.add_conditional_edges("analyze_sentiment", route_sentiment, {"happy": "happy", "sad": "sad"})
builder.add_edge("happy", END)
builder.add_edge("sad", END)

sentiment_graph = builder.compile()


In [ ]:
# Visualize the graph
display(Image(sentiment_graph.get_graph().draw_mermaid_png()))


In [ ]:
# Example 1: Happy sentence
result = sentiment_graph.invoke({"sentence": "I got the job! I'm so excited!"})
print(f"Sentiment: {result['sentiment']}")
print(f"Message: {result['message']}")


In [ ]:
# Example 2: Sad sentence
result = sentiment_graph.invoke({"sentence": "I failed my exam and feel terrible."})
print(f"Sentiment: {result['sentiment']}")
print(f"Message: {result['message']}")


## Why LangGraph then where I can do everything in LangChain?
- THE central framework from LangChain company.
- Less opinionated, more flexibility. Can still use LangChain within the nodes of a graph.
- Contains many primitives for building AI agents over the next few weeks.